## Model 2

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
import os

### Step 1 Merge feature data and price data

In [2]:
# Paths to the input files
feature_file_path = 'Data assignment 1/Feature data.csv'
prices_file_path = 'Data assignment 1/prices_merged_df_output.csv'

# Read the CSV files into pandas dataframes
feature_data_df = pd.read_csv(feature_file_path)
prices_data_df = pd.read_csv(prices_file_path)

# Feature scaling, same as model 1
scaler_standard = StandardScaler()

feature_data_df['Mean wind speed'] = scaler_standard.fit_transform(feature_data_df[['Mean wind speed']])
feature_data_df['Maximum temperature'] = scaler_standard.fit_transform(feature_data_df[['Maximum temperature']])

feature_data_df['Wind direction sin'] = np.sin(np.deg2rad(feature_data_df['Mean wind direction']))
feature_data_df['Wind direction cos'] = np.cos(np.deg2rad(feature_data_df['Mean wind direction']))

nominal_capacity = 30000
feature_data_df['AKI Kalby Active Power'] = feature_data_df['AKI Kalby Active Power'] / nominal_capacity

feature_data_df = feature_data_df.drop('Mean wind direction', axis=1)

feature_data_df['datetime'] = pd.to_datetime(feature_data_df['datetime'])
prices_data_df['HourDK'] = pd.to_datetime(prices_data_df['HourDK'])

# Merge the two datasets on their datetime columns, using an inner join to keep only matching rows
merged_df = pd.merge(feature_data_df, prices_data_df, left_on='datetime', right_on='HourDK', how='inner')

# Drop the redundant 'HourDK' column from the merged dataframe
merged_df = merged_df.drop(columns=['HourDK'])

# Define the output directory and file name
output_dir = 'Data assignment 1'
output_path = os.path.join(output_dir, 'Model2data.csv')

# Save the merged dataframe to a CSV file
merged_df.to_csv(output_path, index=False)


### Step 2

### Classification

In [3]:
data = pd.read_csv('Data assignment 1/Model2data.csv')
data['datetime'] = pd.to_datetime(data['datetime'])

In [4]:
# Step 1: Define the hourly classification rule
def classify_action(row):
    if row['DA_PriceEUR'] > row['BalancingPriceUpEUR'] and row['DA_PriceEUR'] > row['BalancingPriceDownEUR']:
        return 1  # Sell more in day-ahead market
    elif row['DA_PriceEUR'] < row['BalancingPriceUpEUR'] or row['DA_PriceEUR'] < row['BalancingPriceDownEUR']:
        return 2  # Adjust in balancing market (buy/sell)
    else:
        return 3  # Balance between markets

# Apply the classification rule on an hourly basis
data['action_class'] = data.apply(classify_action, axis=1)

In [5]:
# Step 2: Define the objective function for hourly optimization (used for class 3)
def objective_function(predicted_p, actual_p, da_price, up_price, down_price):
    z_down = actual_p - predicted_p
    z_up = predicted_p - actual_p
    
    # Ensure non-negative values for z_down and z_up
    z_down = max(0, z_down)
    z_up = max(0, z_up)
    
    # Return negative to turn maximization problem into a minimization for scipy.optimize
    return -(da_price * predicted_p + (down_price * z_down - up_price * z_up))

In [6]:
# Step 3: Optimization for each hour depending on the action_class
def optimize_hourly_energy_production(row, nominal_capacity):
    actual_p = float(row['AKI Kalby Active Power'])  # Actual power production
    da_price = float(row['DA_PriceEUR'])  # Day-ahead price
    up_price = float(row['BalancingPriceUpEUR'])  # Up-regulation price
    down_price = float(row['BalancingPriceDownEUR'])  # Down-regulation price
    action_class = row['action_class']  # Classification

    # For class 1: Maximize day-ahead market (phat = nominal_capacity)
    if action_class == 1:
        return nominal_capacity
    
    # For class 2: Minimize production (phat = 0)
    elif action_class == 2:
        return 0
    
    # For class 3: Balance between day-ahead and balancing market (optimize phat)
    else:
        initial_predicted_p = actual_p  # Use actual power as an initial guess

        # Constraints: 0 < predicted_p <= nominal_capacity
        constraints = [{'type': 'ineq', 'fun': lambda p: p},  # predicted_p >= 0
                       {'type': 'ineq', 'fun': lambda p: nominal_capacity - p}]  # predicted_p <= nominal_capacity
        
        # Run the optimization for class 3
        result = minimize(objective_function, initial_predicted_p,
                          args=(actual_p, da_price, up_price, down_price),
                          constraints=constraints, method='SLSQP')
        
        # If optimization succeeds, return the optimal predicted_p, else fallback to initial guess
        return result.x[0] if result.success else initial_predicted_p
    
# Apply optimization for each hour
data['optimized_predicted_power'] = data.apply(lambda row: optimize_hourly_energy_production(row, nominal_capacity), axis=1)

In [7]:
# Step 4: Calculate hourly revenue based on the optimized predicted power
def calculate_hourly_revenue(row):
    actual_p = row['AKI Kalby Active Power']
    predicted_p = row['optimized_predicted_power']
    da_price = row['DA_PriceEUR']
    up_price = row['BalancingPriceUpEUR']
    down_price = row['BalancingPriceDownEUR']
    
    z_down = max(0, actual_p - predicted_p)
    z_up = max(0, predicted_p - actual_p)
    
    # Revenue is computed based on the objective function
    revenue = da_price * predicted_p + (down_price * z_down - up_price * z_up)
    return revenue

data['hourly_revenue'] = data.apply(calculate_hourly_revenue, axis=1)

In [8]:
# Step 5: Aggregate hourly revenue into daily revenue
data['date'] = data['datetime'].dt.date  # Extract the date
daily_revenue = data.groupby('date')['hourly_revenue'].sum().reset_index()

# Output only phat (optimized_predicted_power) for each hour and daily revenue
hourly_phat = data[['datetime', 'optimized_predicted_power']]  # phat for each hour

# Display the results
hourly_phat.head(), daily_revenue.head()


(             datetime  optimized_predicted_power
 0 2022-01-01 00:00:00                    30000.0
 1 2022-01-01 01:00:00                    30000.0
 2 2022-01-01 02:00:00                    30000.0
 3 2022-01-01 03:00:00                        0.0
 4 2022-01-01 04:00:00                    30000.0,
          date  hourly_revenue
 0  2022-01-01        0.074067
 1  2022-01-02        0.298774
 2  2022-01-03       -0.006036
 3  2022-01-04       12.240022
 4  2022-01-05        0.959337)

In [9]:
# Calculate total annual revenue
total_annual_revenue = daily_revenue['hourly_revenue'].sum()

# Output total annual revenue
print(f"Total Annual Revenue: {total_annual_revenue}")

Total Annual Revenue: 1503952.481695869
